##### Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Customer Financial Profiler

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Customer_financial_profiler.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

<!-- Community Contributor Badge -->
<table>
  <tr>
    <!-- Author Avatar Cell -->
    <td bgcolor="#d7e6ff">
      <a href="https://github.com/sharathrushi" target="_blank" title="View Sharath Rushi's profile on GitHub">
        <img src="https://github.com/sharathrushi.png?size=100"
             alt="Sharath's GitHub avatar"
             width="100"
             height="100">
      </a>
    </td>
    <!-- Text Content Cell -->
    <td bgcolor="#d7e6ff">
      <h2><font color='black'>This notebook was contributed by <a href="https://github.com/sharathrushi" target="_blank"><font color='#217bfe'><strong>Sharath Rushi</strong></font></a>.</font></h2>
      <h5><font color='black'><a href="https://github.com/sharathrushi" target="_blank">
      <!-- Footer -->
      <font color='black'><small><em>Have a cool Gemini example? Feel free to <a href="https://github.com/google-gemini/cookbook/blob/main/CONTRIBUTING.md" target="_blank"><font color="#078efb">share it too</font></a>!</em></small></font>
    </td>
  </tr>
</table>

# Overview
This tutorial demonstrates estimating customer financial status
Given a customer's financial documents such as payslips, rental agreements, house valuation, shares

Estimating customer monthly income, movable assets, immovable assets

This is helpful for financial institutions to check for loan eligibility, estimating customer's value

## Setup

### Install SDK

In [2]:
%pip install -U -q "google-genai>=1.0.0"  # Install the Python SDK

# Always set at least 1.0.0 as the minimal version as there were breaking
# changes through the previous versions
# Of course, if your notebook uses a new feature and needs a more recent
# version, set the right minimum version to indicate when the feature was
# introduced.
# Always test your notebook with that fixed version (eg. '==1.0.0') to make.
# sure it's really the minimum version.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 23.3 MB/s eta 0:00:00


In [3]:
# This step might ask you to restart the session for the installed packages to be reflected

%pip install -U -q pymupdf nougat-ocr transformers "albumentations==1.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) quickstart for an example.

In [4]:
from google.colab import userdata
from google import genai
from google.genai import types

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [5]:
MODEL_ID = "gemini-3-flash-preview" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-3-flash-preview", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

# Ideally order the model by "cabability" ie. generation then within generation
# 8b/flash-lite then flash then pro

# Configurable params

In [6]:
# Change this to the person you are interested in and upload their financial documents
person_of_interest = "Jesse Nathan"

# Loading necessary sample files
##### Below code downloads the sample input files to session storage
##### This requires user to authenticate
##### Note: These files get deleted after session expires

In [7]:
from google.colab import auth
auth.authenticate_user()

In [8]:
!gcloud storage cp --recursive gs://ksharathrushik-genai/gemini-cookbook-customer-financial-profiler/* .

Copying gs://ksharathrushik-genai/gemini-cookbook-customer-financial-profiler/Jesse Nathan_1.pdf to file://./Jesse Nathan_1.pdf
Copying gs://ksharathrushik-genai/gemini-cookbook-customer-financial-profiler/Jesse Nathan_2.pdf to file://./Jesse Nathan_2.pdf
Copying gs://ksharathrushik-genai/gemini-cookbook-customer-financial-profiler/Jesse Nathan_3.pdf to file://./Jesse Nathan_3.pdf

Average throughput: 13.3MiB/s


To take a quick anonymous survey, run:
  $ gcloud survey



# Imports and files

In [9]:
import io
import os
import json
import re
import torch

In [10]:
grounding_tool = types.Tool(google_search=types.GoogleSearch())
config = types.GenerateContentConfig(tools=[grounding_tool])

In [11]:
from transformers import AutoProcessor, VisionEncoderDecoderModel
all_files = os.listdir('.')
person_of_interest_files = [f for f in all_files if f.startswith(person_of_interest)]
nougat_model_id = "facebook/nougat-small"
processor = AutoProcessor.from_pretrained(nougat_model_id, do_crop_margin=False)
model_base = VisionEncoderDecoderModel.from_pretrained(nougat_model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The image processor of type `NougatImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/484 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

# AI ML Checks
Checks whether the person is linked to illegal activities, terrorism or money laundering

In [12]:
# @title Helper function to add citations
def add_citations(response):
    text = response.text

    # Check if grounding metadata exists and has supports/chunks
    if not response.candidates or not response.candidates[0].grounding_metadata:
        print("No grounding metadata available. Returning original text.")
        return text

    grounding_metadata = response.candidates[0].grounding_metadata
    supports = grounding_metadata.grounding_supports
    chunks = grounding_metadata.grounding_chunks

    if not supports or not chunks:
        print("No grounding supports or chunks available. Returning original text.")
        return text

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = ", ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text

In [13]:
def aml_screening_with_grounding(name):
    prompt = f"""
    Based on recent news and publicly available information, is there any credible indication that {name} is involved in money laundering, terrorism financing, or other illegal activities?
    Return 'True' if there are clear red flags or strong suspicious mentions; otherwise, return 'False'. Respond with only 'True' or 'False'.
    """

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=config,
    )
    print(f"Raw response from Google: {response.text}")

    # Add citations for transparency
    text_with_citations = add_citations(response)
    print(f"Grounding analysis for {name}:\n{text_with_citations}")

    # Convert model's response to boolean
    if response.text:
      genai_analysis = response.text.strip().lower()
      if "true" in genai_analysis:
        return True
      else:
        return False
    else:
        return False

In [14]:
red_flags = aml_screening_with_grounding(person_of_interest)
print(f"Red flags for {person_of_interest}: {red_flags}")

Raw response from Google: False
No grounding supports or chunks available. Returning original text.
Grounding analysis for Jesse Nathan:
False
Red flags for Jesse Nathan: False


# Processing Source of Wealth Files
##### Below code loads pdf, extracts text and generates the financial profile of the customer
##### Note: It also supports image or scanned pdf documents

In [15]:
import pymupdf as fitz
from PIL import Image

def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page_num, page in enumerate(doc):
        page_text_fitz = page.get_text()

        # If fitz extracts no text, assume it's an image-based PDF and use Nougat
        if not page_text_fitz.strip():
            # Render page to an image (high resolution for OCR)
            # Using a matrix to increase resolution (e.g., 2x2 or 3x3) can improve OCR accuracy.
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2)) # 2x resolution
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

            # Process image with Nougat
            pixel_values = processor(images=img, return_tensors="pt", do_crop_margin=False, do_thumbnail=False, do_align_long_axis=False).pixel_values

            # Ensure pixel_values are float32 for the model
            if pixel_values.dtype != torch.float32:
                pixel_values = pixel_values.float()

            # Generate text using the Nougat model. Removed decoder_input_ids as it's not needed for starting generation from image.
            outputs = model_base.generate(
                pixel_values,
                min_length=1,
                max_new_tokens=1024, # Reduced max_new_tokens to prevent index out of range error
                bad_words_ids=[[processor.tokenizer.unk_token_id]],
            )
            page_text_ocr = processor.batch_decode(outputs, skip_special_tokens=True)[0]
            full_text += page_text_ocr + "\n\n" # Add some separation between pages
        else:
            full_text += page_text_fitz + "\n\n" # Add some separation between pages
    return full_text

In [16]:
json_schema = """
    {
      "type": "object",
      "properties": {
        "all_sources_primarily_about_target_name": { "type": "string" },
        "aggregated_values": {
          "type": "object",
          "properties": {
            "monthly_income": {
              "type": "object",
              "properties": {
                "value": { "type": "number" },
                "currency": { "type": "string" },
                "period": { "type": "string" }
              },
              "required": ["value", "currency", "period"]
            },
            "immovable_assets": {
              "type": "array",
              "items": {
                "type": "object",
                "properties": {
                  "description": { "type": "string" },
                  "value": { "type": "number" },
                  "currency": { "type": "string" },
                  "type": { "type": "string" },
                  "valuation_basis": { "type": "string" },
                  "age_of_property_years": { "type": "number" }
                },
                "required": ["description", "value", "currency", "type", "valuation_basis", "age_of_property_years"]
              }
            },
            "movable_assets": {
              "type": "object",
              "properties": {
                "stocks": {
                  "type": "array",
                  "items": {
                    "type": "object",
                    "properties": {
                      "company_name": { "type": "string" },
                      "number_of_shares": { "type": "number" },
                      "par_value_per_share": { "type": "number" },
                      "currency_per_share": { "type": "string" },
                      "total_value": { "type": "number" },
                      "total_value_currency": { "type": "string" },
                      "owner": { "type": "string" }
                    },
                    "required": ["company_name", "number_of_shares", "par_value_per_share", "currency_per_share", "total_value", "total_value_currency", "owner"]
                  }
                },
                "cash": {
                  "type": "object",
                  "properties": {
                    "value": { "type": "number" },
                    "currency": { "type": "string" },
                    "note": { "type": "string" }
                  },
                  "required": ["value", "currency", "note"]
                }
              },
              "required": ["stocks", "cash"]
            }
          },
          "required": ["monthly_income", "immovable_assets", "movable_assets"]
        }
      },
      "required": ["all_sources_primarily_about_target_name", "aggregated_values"]
    }
    """

In [17]:
def verify_and_extract(text, target_name):
    prompt = f"""
    Analyze the following text from multiple sources regarding {target_name}.
    1. Verify if all sources document is primarily about {target_name}. (Yes/No)
    2. Aggregate the values to retrieve the following information:
        - Monthly Income
        - Immovable Assets (Real Estate)
        - Movable Assets (Cash/Stocks)
    3. Format as JSON.
    Text: {text[:10000]} # Truncate for token limits
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": json.loads(json_schema)
            }
        )
    return response.text

In [18]:
combined_text = ""
for file in person_of_interest_files:
    text = pdf_to_text(file)
    print(f"Extracting information from {file}")
    combined_text += text + "\n" # Added newline character
result = verify_and_extract(combined_text, person_of_interest)
print(result)

Extracting information from Jesse Nathan_1.pdf
Extracting information from Jesse Nathan_2.pdf
Extracting information from Jesse Nathan_3.pdf
{"all_sources_primarily_about_target_name":"Yes","aggregated_values":{"monthly_income":{"value":8000,"currency":"USD","period":"Monthly"},"immovable_assets":[],"movable_assets":{"stocks":[{"company_name":"Everest Financial Group Inc.","number_of_shares":250,"par_value_per_share":1,"currency_per_share":"USD","total_value":250,"total_value_currency":"USD","owner":"Jesse Nathan"}],"cash":{"value":0,"currency":"USD","note":"No cash or bank account balances were specified in the documents."}}}}


## Next steps
1. Token & Cost Management

    Token Counts: Use model.count_tokens(prompt) before calling the API.

    Billing: Gemini 1.5 Flash is significantly cheaper than Pro. For high-volume PDF parsing, use Flash.

    Reduction: Implement Semantic Chunking. Instead of sending a 100-page PDF, use a lightweight NLP tool (like spaCy) to find pages containing "Balance Sheet" or "Assets" and only send those pages to the LLM.

2. Metrics & Benchmarking
Metric	Description
Extraction Accuracy	Compare LLM output against a manually labeled "Golden Dataset."
Latency	Time taken from PDF upload to final report (Target: < 30s).
F1 Score	For NER (Named Entity Recognition) of asset values.
3. Hallucination Check & Monitoring

    Self-Reflection: Ask the model to "Provide the exact quote from the text where you found this asset value." If it can't, flag it as a potential hallucination.

    NLI (Natural Language Inference): Use a smaller model to check if the generated "Summary" is logically entailed by the "Source Text."

    Monitoring: Use tools like Arize Phoenix or LangSmith to track drift and "faithfulness" scores in production.

4. Human-in-the-Loop (HITL)

In financial compliance, AI should never make the final "Reject" decision.

    Confidence Thresholds: If the LLM confidence is <0.85, route the case to a compliance officer.

    UI Annotation: Highlight the source PDF text in a dashboard so the human can quickly verify the AI's extraction.

5. NLP Alternatives (Non-GenAI)

To reduce costs or increase speed, use:

    Entity Extraction: spaCy or Hugging Face FinBERT (fine-tuned for finance).

    Pattern Matching: Regular Expressions (Regex) for specific formats like CIBIL scores or Currency amounts.

    Classification: XGBoost on TF-IDF vectors to categorize documents before they reach the LLM.